In [13]:
# Code for generating links for getting individual images of pages

ZOOM = 1

imgurl_pre = "https://repository.ou.edu/adore-djatoka/resolver?url_ver=Z39.88-2004&rft_id=https://repository.ou.edu/uuid/"
imgurl_post = "/datastream/JP2/view&svc_id=info:lanl-repo/svc/getRegion&svc_val_fmt=info:ofi/fmt:kev:mtx:jpeg2000&svc.format=image/jpeg&svc.level=0"
imgurl_post += str(ZOOM)
imgurl_post += "&svc.rotate=0"

noimg_uuid = "99f5f78c-24f8-5042-beb8-4decbd43ec81"
img_uuid = "f32fc0a3-d585-5912-a016-5cb56f740cf3"

print(imgurl_pre+noimg_uuid+imgurl_post)
print(imgurl_pre+img_uuid+imgurl_post)

https://repository.ou.edu/adore-djatoka/resolver?url_ver=Z39.88-2004&rft_id=https://repository.ou.edu/uuid/99f5f78c-24f8-5042-beb8-4decbd43ec81/datastream/JP2/view&svc_id=info:lanl-repo/svc/getRegion&svc_val_fmt=info:ofi/fmt:kev:mtx:jpeg2000&svc.format=image/jpeg&svc.level=01&svc.rotate=0
https://repository.ou.edu/adore-djatoka/resolver?url_ver=Z39.88-2004&rft_id=https://repository.ou.edu/uuid/f32fc0a3-d585-5912-a016-5cb56f740cf3/datastream/JP2/view&svc_id=info:lanl-repo/svc/getRegion&svc_val_fmt=info:ofi/fmt:kev:mtx:jpeg2000&svc.format=image/jpeg&svc.level=01&svc.rotate=0


In [2]:
## Start extract-figures.py code

import argparse
import json
import os
import sys

import cv2
import numpy

from image_mining.figure_extraction import FigureExtractor
from image_mining.utils import open_image

In [50]:
params = {
"canny_threshold": 0,
"erosion_element": "rectangle",
"erosion_size": 12,
"dilation_element": "rectangle",
"dilation_size": 2,
# "min_area": ,
# "min_height": ,
# "max_height": ,
# "min_width": ,
# "max_width": ,
}

INTERACTIVE = False
FILES = ["./testimages/img5.jpg"]
SAVE_JSON = False
output_dir = os.getcwd()

extractor = FigureExtractor(**params)

if INTERACTIVE:
    display_images(extractor, FILES)
else:
    for f in FILES:
        try:
            base_name, source_image = open_image(f)
        except StandardError as exc:
            print >>sys.stderr, exc
            continue

        output_base = os.path.join(output_dir, base_name)

        print("Processing %s" % f)

        boxes = []

        for i, bbox in enumerate(extractor.find_figures(source_image), 1):
            extracted = source_image[bbox.image_slice]
            extract_filename = os.path.join(output_dir, "%s-%d.jpg" % (output_base, i))
            print("\tSaving %s" % extract_filename)
            cv2.imwrite(extract_filename, extracted)

            boxes.append(bbox.as_dict())

        if SAVE_JSON and boxes:
            json_data = {"source_image": {"filename": f,
                                          "dimensions": {"width": source_image.shape[1],
                                                         "height": source_image.shape[0]}},
                         "regions": boxes}

            json_filename = os.path.join(output_dir, "%s.json" % output_base)
            with open(json_filename, "wb") as json_f:
                json.dump(json_data, json_f, allow_nan=False)
            print("\tSaved extract information to %s" % json_filename)

Processing ./testimages/img5.jpg
	Saving /home/eth/Dropbox/School/LibRepo/image-mining/img5-1.jpg
